# Polar Coding Notebook

In [1]:
# Polar code (natural-order recursive encoder + SC decoder with exact LLR f)
# Restart kernel, paste this cell, and run.

import numpy as np
import matplotlib.pyplot as plt

# -------------------------
# helpers
# -------------------------
def is_power_of_two(n):
    return (n & (n-1)) == 0 and n > 0

def bit_reverse_indices(N):
    n = int(np.log2(N))
    br = np.zeros(N, dtype=int)
    for i in range(N):
        br[i] = int(format(i, 'b').zfill(n)[::-1], 2)
    return br

def bhattacharyya_parameter(p, N):
    Z = np.array([2.0 * np.sqrt(p * (1.0 - p))])
    while len(Z) < N:
        Z_next = []
        for z in Z:
            Z_next.append(2*z - z*z)
            Z_next.append(z*z)
        Z = np.array(Z_next)
    return Z

def select_frozen_bits_for_recursive_encoder(N, K, p):
    """
    Select K best logical channels and map to natural-order positions
    so recursive encoder and SC decoder agree.
    """
    Z = bhattacharyya_parameter(p, N)
    best_logical = np.argsort(Z)[:K]
    br = bit_reverse_indices(N)
    info_positions = br[best_logical]   # logical -> natural
    frozen_mask = np.ones(N, dtype=bool)
    frozen_mask[info_positions] = False
    return frozen_mask, np.sort(info_positions)

# -------------------------
# recursive polar transform (natural order)
# -------------------------
def polar_transform(u):
    N = len(u)
    if N == 1:
        return u.copy()
    u_even = u[0:N:2]
    u_odd  = u[1:N:2]
    upper = polar_transform((u_even ^ u_odd) % 2)
    lower = polar_transform(u_odd)
    return np.concatenate([upper, lower]).astype(np.int8)

def polar_encode_recursive(msg, info_positions, N, u_frozen=None):
    if u_frozen is None:
        u_frozen = np.zeros(N, dtype=np.int8)
    u = np.array(u_frozen, dtype=np.int8)
    u[np.sort(info_positions)] = np.array(msg, dtype=np.int8)
    x = polar_transform(u)
    return x.astype(np.int8), u.astype(np.int8)

# -------------------------
# channel models & LLRs
# -------------------------
def bsc(x, p):
    flips = (np.random.rand(len(x)) < p).astype(np.int8)
    return (x ^ flips).astype(np.int8)

def bsc_llr(y, p, eps=1e-12):
    # LLR for BSC: log((1-p)/p) * (1 - 2y)
    p_safe = min(max(float(p), eps), 1.0 - eps)
    alpha = np.log((1.0 - p_safe) / p_safe)
    return (1 - 2*y) * alpha

import numpy as np

# Stable exact f using tanh/atanh (fixed np.errstate usage)
def f_exact(a, b, eps=1e-12):
    """
    Exact LLR combination f(a,b) = 2*atanh(tanh(a/2)*tanh(b/2))
    Implemented in a numerically stable vectorized way.
    """
    A = np.array(a, dtype=float)
    B = np.array(b, dtype=float)
    ta = np.tanh(A / 2.0)
    tb = np.tanh(B / 2.0)
    t = ta * tb
    # clamp t away from ±1
    t = np.clip(t, -1.0 + eps, 1.0 - eps)
    # 2*atanh(t) = log((1+t)/(1-t)); use log1p for stability
    with np.errstate(divide='ignore', over='ignore', invalid='ignore'):
        L = np.log1p(t) - np.log1p(-t)
    return L

# Redefine the SC decoder that uses f_exact (so the current kernel uses the fixed version)
def successive_cancellation_decode_exact(llr, frozen, u_frozen=None):
    import numpy as _np
    if u_frozen is None:
        u_frozen = _np.zeros(len(frozen), dtype=_np.int8)
    N = len(llr)
    frozen = _np.array(frozen, dtype=bool)
    u_frozen = _np.array(u_frozen, dtype=_np.int8)
    llr = _np.array(llr, dtype=float)

    def sc_rec(llr_sub, frozen_sub, u_frozen_sub):
        n = len(llr_sub)
        if n == 1:
            if frozen_sub[0]:
                return _np.array([u_frozen_sub[0]], dtype=_np.int8)
            else:
                return _np.array([0 if llr_sub[0] >= 0 else 1], dtype=_np.int8)
        n2 = n // 2
        a = llr_sub[:n2]
        b = llr_sub[n2:]
        # use fixed exact f
        f_llr = f_exact(a, b)
        uhat_upper = sc_rec(f_llr, frozen_sub[:n2], u_frozen_sub[:n2])
        sign_term = (1 - 2 * uhat_upper)
        g_llr = b + sign_term * a
        uhat_lower = sc_rec(g_llr, frozen_sub[n2:], u_frozen_sub[n2:])
        u_upper = (uhat_upper ^ uhat_lower).astype(_np.int8)
        return _np.concatenate([u_upper, uhat_lower]).astype(_np.int8)

    return sc_rec(llr, frozen, u_frozen)

# If you used monte_carlo_polar_exact earlier, re-run it now (example quick sanity):
# ber0, bler0 = monte_carlo_polar_exact(N=32, K=16, p_design=0.01, p_channel=0.0, trials=200)
# print("No-noise check:", ber0, bler0)


# -------------------------
# Monte-Carlo using exact f + natural-order pairing
# -------------------------
def monte_carlo_polar_exact(N=128, K=None, p_design=0.05, p_channel=None, trials=1000, verbose=False):
    if not is_power_of_two(N):
        raise ValueError("N must be power of two")
    if K is None:
        K = N // 2
    if p_channel is None:
        p_channel = p_design

    frozen_mask, info_idx = select_frozen_bits_for_recursive_encoder(N, K, p_design)
    info_idx_sorted = np.sort(info_idx)
    u_frozen = np.zeros(N, dtype=np.int8)

    bit_err = 0
    block_err = 0
    for t in range(trials):
        msg = np.random.randint(0, 2, size=K).astype(np.int8)
        x, u = polar_encode_recursive(msg, info_idx_sorted, N, u_frozen=u_frozen)
        y = bsc(x, p_channel)
        llr = bsc_llr(y, p_channel)
        uhat = successive_cancellation_decode_exact(llr, frozen_mask, u_frozen=u_frozen)
        msg_hat = uhat[info_idx_sorted]
        this_bit_err = int(np.sum(msg_hat != msg))
        bit_err += this_bit_err
        block_err += int(this_bit_err > 0)
        if verbose and (t % max(1, trials // 10) == 0):
            print(f"trial {t}: bit_err={bit_err}, block_err={block_err}")
    ber = bit_err / (trials * K)
    bler = block_err / trials
    return ber, bler

# -------------------------
# Quick sanity checks
# -------------------------
if __name__ == "__main__":
    print("Running sanity checks (exact f, natural-order encoder/decoder).")
    N = 32
    K = N // 2
    ber0, bler0 = monte_carlo_polar_exact(N=N, K=K, p_design=0.01, p_channel=0.0, trials=200)
    print("No-noise check: BER =", ber0, "BLER =", bler0)
    ber50, bler50 = monte_carlo_polar_exact(N=N, K=K, p_design=0.5, p_channel=0.5, trials=200)
    print("p=0.5 check: BER =", ber50, "BLER =", bler50)


Running sanity checks (exact f, natural-order encoder/decoder).
No-noise check: BER = 0.4696875 BLER = 1.0
p=0.5 check: BER = 0.50875 BLER = 1.0


In [2]:
# FINAL MINIMAL SELF-CONTAINED DIAGNOSTIC
# Restart kernel, paste this single cell and run it.

import numpy as np
from math import log2

# ---- definitions (all fresh) ----
def polar_transform(u):
    N = len(u)
    if N == 1:
        return u.copy()
    u_even = u[0:N:2]
    u_odd  = u[1:N:2]
    upper = polar_transform((u_even ^ u_odd) % 2)
    lower = polar_transform(u_odd)
    return np.concatenate([upper, lower]).astype(np.int8)

def kron_F_power(N):
    assert (N & (N-1)) == 0 and N>0
    F = np.array([[1,0],[1,1]], dtype=int)
    n = int(log2(N))
    G = np.array([[1]], dtype=int)
    for _ in range(n):
        G = np.kron(G, F)
    return G % 2

def f_exact(a,b,eps=1e-12):
    A = np.array(a, dtype=float); B = np.array(b, dtype=float)
    ta = np.tanh(A/2.0); tb = np.tanh(B/2.0)
    t = np.clip(ta*tb, -1+eps, 1-eps)
    return np.log1p(t) - np.log1p(-t)  # = 2*atanh(t)

def sc_decode_exact_instrumented(llr, frozen, u_frozen=None):
    import numpy as _np
    if u_frozen is None:
        u_frozen = _np.zeros(len(frozen), dtype=_np.int8)
    llr = _np.array(llr, dtype=float)
    frozen = _np.array(frozen, dtype=bool)
    u_frozen = _np.array(u_frozen, dtype=_np.int8)
    def rec(llr_sub, frozen_sub, u_frozen_sub, depth=0, path="root"):
        indent = "  "*depth
        n = len(llr_sub)
        print(f"{indent}{path}: n={n}, llr_sub={llr_sub.tolist()}, frozen_sub={frozen_sub.tolist()}")
        if n==1:
            if frozen_sub[0]:
                print(f"{indent} => leaf frozen -> {u_frozen_sub[0]}")
                return _np.array([u_frozen_sub[0]],dtype=_np.int8)
            dec = 0 if llr_sub[0] >= 0 else 1
            print(f"{indent} => leaf decide -> {dec}")
            return _np.array([dec], dtype=_np.int8)
        n2 = n//2
        a = llr_sub[:n2]; b = llr_sub[n2:]
        f = f_exact(a,b)
        print(f"{indent} f: a={a.tolist()}, b={b.tolist()} -> f={f.tolist()}")
        u_up = rec(f, frozen_sub[:n2], u_frozen_sub[:n2], depth+1, path=path+"-L")
        print(f"{indent} u_up returned: {u_up.tolist()}")
        sign = (1-2*u_up)
        g = b + sign * a
        print(f"{indent} g: sign={sign.tolist()}, g={g.tolist()}")
        u_low = rec(g, frozen_sub[n2:], u_frozen_sub[n2:], depth+1, path=path+"-R")
        print(f"{indent} u_low returned: {u_low.tolist()}")
        combined = _np.concatenate([(u_up ^ u_low).astype(_np.int8), u_low]).astype(_np.int8)
        print(f"{indent} combined: {combined.tolist()}")
        return combined
    return rec(llr, frozen, u_frozen, depth=0, path="root")

# ---- test parameters ----
N = 4
K = N//2
p_design = 0.05

# compute logical reliabilities and map to natural-order positions
def bhattacharyya(p,N):
    Z = np.array([2.0*np.sqrt(p*(1-p))])
    while len(Z)<N:
        Zn=[]
        for z in Z:
            Zn.append(2*z - z*z); Zn.append(z*z)
        Z=np.array(Zn)
    return Z

Z = bhattacharyya(p_design, N)
logical_best = np.argsort(Z)[:K]
br = np.array([int(format(i,'b').zfill(int(log2(N)))[::-1],2) for i in range(N)])
info_nat = np.sort(br[logical_best])

# build u (natural order) with deterministic message
msg = np.array([0,1], dtype=np.int8)
u = np.zeros(N, dtype=np.int8); u[info_nat] = msg
x = polar_transform(u)
Gnat = kron_F_power(N)
x_g = (u.astype(int) @ Gnat.astype(int)) % 2

print("N,K:",N,K)
print("logical_best:", logical_best.tolist())
print("bit-reverse br:", br.tolist())
print("info positions (natural):", info_nat.tolist())
print("u natural:", u.tolist())
print("x (transform):", x.tolist())
print("x (u@Gnat):", x_g.tolist(), "match:", np.array_equal(x,x_g))

# generate noiseless LLRs
eps = 1e-12
alpha = np.log((1.0-eps)/eps)
y = x.copy()
llr = (1-2*y) * alpha
print("llr:", llr.tolist())
print("frozen_mask (natural order):", (~np.isin(np.arange(N), info_nat)).tolist())

print("\nRun instrumented exact-SC now:")
uhat = sc_decode_exact_instrumented(llr, (~np.isin(np.arange(N), info_nat)), u_frozen=np.zeros(N,dtype=np.int8))
print("\nfinal uhat:", uhat.tolist())
print("expected u:", u.tolist())
print("match?", np.array_equal(uhat, u))


N,K: 4 2
logical_best: [3, 2]
bit-reverse br: [0, 2, 1, 3]
info positions (natural): [1, 3]
u natural: [0, 0, 0, 1]
x (transform): [1, 1, 1, 1]
x (u@Gnat): [1, 1, 1, 1] match: True
llr: [-27.63102111592755, -27.63102111592755, -27.63102111592755, -27.63102111592755]
frozen_mask (natural order): [True, False, True, False]

Run instrumented exact-SC now:
root: n=4, llr_sub=[-27.63102111592755, -27.63102111592755, -27.63102111592755, -27.63102111592755], frozen_sub=[True, False, True, False]
 f: a=[-27.63102111592755, -27.63102111592755], b=[-27.63102111592755, -27.63102111592755] -> f=[26.937840546492907, 26.937840546492907]
  root-L: n=2, llr_sub=[26.937840546492907, 26.937840546492907], frozen_sub=[True, False]
   f: a=[26.937840546492907], b=[26.937840546492907] -> f=[26.244693365930964]
    root-L-L: n=1, llr_sub=[26.244693365930964], frozen_sub=[True]
     => leaf frozen -> 0
   u_up returned: [0]
   g: sign=[1], g=[53.875681092985815]
    root-L-R: n=1, llr_sub=[53.875681092985815]

In [1]:
# LAST-ALTERNATIVE: recursive encoder + exact-f SC decoder
# Frozen indices chosen directly by Bhattacharyya order (no bit-reversal)
import numpy as np
from math import log2

def polar_transform(u):
    N = len(u)
    if N == 1:
        return u.copy()
    u_even = u[0:N:2]
    u_odd  = u[1:N:2]
    return np.concatenate([polar_transform((u_even ^ u_odd) % 2), polar_transform(u_odd)]).astype(np.int8)

def bhattacharyya_parameter(p, N):
    Z = np.array([2.0 * np.sqrt(p * (1.0 - p))])
    while len(Z) < N:
        Z_next = []
        for z in Z:
            Z_next.append(2*z - z*z)
            Z_next.append(z*z)
        Z = np.array(Z_next)
    return Z

def select_frozen_direct(N, K, p):
    # Directly pick the K best indices from Z and place them in those natural positions
    Z = bhattacharyya_parameter(p, N)
    best = np.argsort(Z)[:K]
    frozen = np.ones(N, dtype=bool)
    frozen[best] = False
    return frozen, np.sort(best)

def f_exact(a,b,eps=1e-12):
    A = np.array(a, dtype=float); B = np.array(b, dtype=float)
    ta = np.tanh(A/2.0); tb = np.tanh(B/2.0)
    t = np.clip(ta*tb, -1+eps, 1-eps)
    return np.log1p(t) - np.log1p(-t)

def sc_decode_exact(llr, frozen, u_frozen=None):
    import numpy as _np
    if u_frozen is None:
        u_frozen = _np.zeros(len(frozen), dtype=_np.int8)
    llr = _np.array(llr, dtype=float)
    frozen = _np.array(frozen, dtype=bool)
    u_frozen = _np.array(u_frozen, dtype=_np.int8)
    def rec(llr_sub, frozen_sub, u_frozen_sub):
        n = len(llr_sub)
        if n == 1:
            if frozen_sub[0]:
                return _np.array([u_frozen_sub[0]], dtype=_np.int8)
            else:
                return _np.array([0 if llr_sub[0] >= 0 else 1], dtype=_np.int8)
        n2 = n//2
        a = llr_sub[:n2]; b = llr_sub[n2:]
        f = f_exact(a,b)
        u_up = rec(f, frozen_sub[:n2], u_frozen_sub[:n2])
        sign = (1 - 2 * u_up)
        g = b + sign * a
        u_low = rec(g, frozen_sub[n2:], u_frozen_sub[n2:])
        u_comb_up = (u_up ^ u_low).astype(_np.int8)
        return _np.concatenate([u_comb_up, u_low]).astype(_np.int8)
    return rec(llr, frozen, u_frozen)

def bsc(x,p):
    flips = (np.random.rand(len(x)) < p).astype(np.int8)
    return (x ^ flips).astype(np.int8)

def bsc_llr(y, p, eps=1e-12):
    p_safe = min(max(float(p), eps), 1.0 - eps)
    alpha = np.log((1.0 - p_safe) / p_safe)
    return (1 - 2*y) * alpha

def monte_carlo_try_direct(N=32, K=None, p_design=0.05, p_channel=None, trials=200):
    if K is None:
        K = N//2
    if p_channel is None:
        p_channel = p_design
    frozen, info_idx = select_frozen_direct(N, K, p_design)
    u_frozen = np.zeros(N, dtype=np.int8)
    bit_err = 0
    block_err = 0
    for t in range(trials):
        msg = np.random.randint(0,2,size=K).astype(np.int8)
        u = np.array(u_frozen, dtype=np.int8)
        u[np.sort(info_idx)] = msg
        x = polar_transform(u)
        y = bsc(x, p_channel)
        llr = bsc_llr(y, p_channel)
        uhat = sc_decode_exact(llr, frozen, u_frozen=u_frozen)
        msg_hat = uhat[np.sort(info_idx)]
        be = int(np.sum(msg_hat != msg))
        bit_err += be
        block_err += int(be>0)
    return bit_err/(trials*K), block_err/trials

# Run quick sanity
print("RUNNING: direct-frozen (no bit-reversal mapping) test")
ber0,bler0 = monte_carlo_try_direct(N=32, K=16, p_design=0.01, p_channel=0.0, trials=200)
print("No-noise check:", ber0, bler0)
ber5,bler5 = monte_carlo_try_direct(N=32, K=16, p_design=0.5, p_channel=0.5, trials=200)
print("p=0.5 check:", ber5, bler5)

# Also print tiny N=4 deterministic instrumented outcome for clarity
N=4; K=2
frozen, info_idx = select_frozen_direct(N,K,0.05)
u_frozen = np.zeros(N,dtype=np.int8)
msg = np.array([0,1],dtype=np.int8)
u = np.zeros(N,dtype=np.int8); u[info_idx]=msg
x = polar_transform(u)
y=x.copy(); llr=bsc_llr(y,1e-12)
print("\nN=4 direct-frozen instrumented check")
print("info_idx:", info_idx.tolist(), "u:", u.tolist(), "x:", x.tolist(), "frozen:", frozen.tolist())
print("llr:", llr.tolist())
# instrumented SC decisions:
def sc_print(llr,frozen,u_frozen=None,depth=0,path="root"):
    import numpy as _np
    if u_frozen is None:
        u_frozen = _np.zeros(len(frozen),dtype=_np.int8)
    n=len(llr); indent="  "*depth
    print(f"{indent}{path}: n={n}, llr={llr.tolist()}, frozen={frozen.tolist()}")
    if n==1:
        if frozen[0]:
            print(f"{indent} leaf frozen -> {u_frozen[0]}")
            return _np.array([u_frozen[0]],dtype=_np.int8)
        dec = 0 if llr[0]>=0 else 1
        print(f"{indent} leaf decide -> {dec}")
        return _np.array([dec],dtype=_np.int8)
    n2=n//2
    a=llr[:n2]; b=llr[n2:]
    f = f_exact(a,b)
    print(f"{indent} f: a={a.tolist()}, b={b.tolist()}, f={f.tolist()}")
    uup = sc_print(f,frozen[:n2],u_frozen[:n2],depth+1,path=path+"-L")
    print(f"{indent} uup returned: {uup.tolist()}")
    sign=(1-2*uup); g = b + sign*a
    print(f"{indent} g: sign={sign.tolist()}, g={g.tolist()}")
    ulow = sc_print(g,frozen[n2:],u_frozen[n2:],depth+1,path=path+"-R")
    print(f"{indent} ulow returned: {ulow.tolist()}")
    combined = np.concatenate([(uup ^ ulow).astype(np.int8), ulow]).astype(np.int8)
    print(f"{indent} combined: {combined.tolist()}")
    return combined

uhat = sc_print(llr,frozen,u_frozen)
print("final uhat:", uhat.tolist(), "expected u:", u.tolist(), "match?", np.array_equal(uhat,u))


RUNNING: direct-frozen (no bit-reversal mapping) test
No-noise check: 0.47625 1.0
p=0.5 check: 0.49125 1.0

N=4 direct-frozen instrumented check
info_idx: [2, 3] u: [0, 0, 0, 1] x: [1, 1, 1, 1] frozen: [True, True, False, False]
llr: [-27.63102111592755, -27.63102111592755, -27.63102111592755, -27.63102111592755]
root: n=4, llr=[-27.63102111592755, -27.63102111592755, -27.63102111592755, -27.63102111592755], frozen=[True, True, False, False]
 f: a=[-27.63102111592755, -27.63102111592755], b=[-27.63102111592755, -27.63102111592755], f=[26.937840546492907, 26.937840546492907]
  root-L: n=2, llr=[26.937840546492907, 26.937840546492907], frozen=[True, True]
   f: a=[26.937840546492907], b=[26.937840546492907], f=[26.244693365930964]
    root-L-L: n=1, llr=[26.244693365930964], frozen=[True]
     leaf frozen -> 0
   uup returned: [0]
   g: sign=[1], g=[53.875681092985815]
    root-L-R: n=1, llr=[53.875681092985815], frozen=[True]
     leaf frozen -> 0
   ulow returned: [0]
   combined: [0, 